In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_raw = pd.read_csv("./data/train_raw.csv")
df = pd.read_csv("./data/train.csv")
codebook_df = pd.read_csv("./data/codebook.csv")

/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,86,87,88,89,100,123,124,125,126,127,128,130,131,133,192,193,194,195,204,205,206,207,228,229,230,258,259,260,261,262) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,86,87,88,89,98,99,100,105,124,125,126,127,128,130,131,133,166,171,172,192,193,194,195,204,205,206,207,228,229,230,240,256,257,258,259,260,261,262) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_raw["v5"].unique()

array(['No second ancestry', 'Austrian nfs', 'Hungarian nfs',
       'German nfs', 'Croatian nfs', 'Refusal', 'Polish nfs',
       'Slovene nfs', 'Romanian nfs', 'East and South African nec',
       'Serbian nfs', "Don't know", 'Turkish', 'Bosniak/Bosnian',
       'East European nec', 'Thai', 'Arab nec', 'Italian nfs',
       'Kosovar nfs', 'Spanish nfs', 'Czech nfs', 'Kenyan', 'Slovak nfs',
       'Russian nfs', 'South American nec', 'Bulgarian nfs',
       'Albanian nfs', 'Egyptian', 'Not classifiable', 'Belgian nfs',
       'Roma/Gypsy/Sinti', 'Swiss nfs', 'Peruvian', 'British nfs',
       'Indian nfs', 'Indonesian nfs', 'Walloon', 'Canadian nfs',
       'Swiss Canton', 'Swiss language region nec', 'Tibetan', 'Moldovan',
       'Iraqi', 'French nfs', 'Danish nfs', 'Lebanese', 'Macedonian',
       'Argentinian', 'American nfs', 'Syrian', 'Armenian', 'English',
       'Brazilian', 'No answer', 'Taiwanese Chinese', 'Colombian',
       'Greek nfs', 'Lithuanian nfs',
       'Other North 

In [5]:
df["satisfied"].value_counts()

1    15626
0    14454
Name: satisfied, dtype: int64

In [6]:
df.count()

id           30080
v1           30080
v2           30080
v3           30080
v4           30080
             ...  
v268         28397
v269         24466
v270         19816
cntry        30080
satisfied    30080
Length: 273, dtype: int64

In [7]:
df.iloc[0]

id            9948
v1               2
v2               2
v3              74
v4           11010
             ...  
v268            .a
v269            .a
v270           NaN
cntry           AT
satisfied        0
Name: 0, Length: 273, dtype: object

In [8]:
df_raw.iloc[0]

id                     9948
v1                        2
v2                     Safe
v3                       74
v4             Austrian nfs
                  ...      
v268         Not applicable
v269         Not applicable
v270                    NaN
cntry                    AT
satisfied                 0
Name: 0, Length: 273, dtype: object

In [45]:
k = df_raw.apply(lambda col : pd.Series(["Yes"]).isin(col)).T[0] | df_raw.apply(lambda col : pd.Series(["No"]).isin(col)).T[0]

In [47]:
k[k==True].index

Index(['v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v14', 'v18', 'v21', 'v22', 'v23',
       'v24', 'v43', 'v53', 'v77', 'v105', 'v108', 'v152', 'v157', 'v162',
       'v165', 'v172', 'v173', 'v187', 'v188', 'v221', 'v241', 'v242', 'v243',
       'v248', 'v254', 'v256', 'v257'],
      dtype='object')

In [43]:
df_raw["v105"].value_counts()

No                    21884
Yes to some extent     6090
Yes a lot              2039
Don't know               49
Refusal                  10
No answer                 8
Name: v105, dtype: int64

In [48]:
df[['v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v14', 'v18', 'v21', 'v22', 'v23',
       'v24', 'v43', 'v53', 'v77', 'v105', 'v108', 'v152', 'v157', 'v162',
       'v165', 'v172', 'v173', 'v187', 'v188', 'v221', 'v241', 'v242', 'v243',
       'v248', 'v254', 'v256', 'v257']]

,v6,v7,v8,v9,v10,v11,v14,v18,v21,v22,...,v187,v188,v221,v241,v242,v243,v248,v254,v256,v257
0,2,2,2,2,1,1,1,2,2,2,...,1,.a,2,.a,2,.a,1,2,2,2
1,2,2,2,2,1,2,2,2,2,2,...,1,.a,2,2,1,1,1,2,2,2
2,2,2,1,2,1,.a,2,2,2,.a,...,1,.a,2,.a,2,.a,1,2,2,2
3,2,2,2,2,1,2,2,2,2,.a,...,1,.a,2,.a,2,.a,2,2,2,2
4,2,2,2,2,1,2,1,1,2,.a,...,1,.a,1,.a,2,.a,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30075,2,2,2,2,1,1,2,2,2,2,...,1,.a,2,.a,2,.a,1,2,NaN,2
30076,2,2,2,2,1,2,2,2,2,.a,...,2,2,2,.a,2,.a,2,2,NaN,2
30077,2,2,2,2,1,.a,2,2,2,.a,...,2,2,2,2,1,2,1,2,NaN,2
30078,2,2,2,2,1,2,2,2,2,2,...,2,2,2,.a,2,.a,3,.a,NaN,2


In [57]:
df_raw[['v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v14', 'v18', 'v21', 'v22', 'v23',
       'v24', 'v43', 'v53', 'v77', 'v105', 'v108', 'v152', 'v157', 'v162',
       'v165', 'v172', 'v173', 'v187', 'v188', 'v221', 'v241', 'v242', 'v243',
       'v248', 'v254', 'v256', 'v257']]

,v6,v7,v8,v9,v10,v11,v14,v18,v21,v22,...,v187,v188,v221,v241,v242,v243,v248,v254,v256,v257
0,No,No,No,No,Yes,Yes,Yes,No,No,No,...,Yes,Not applicable,No,Not applicable,No,Not applicable,Yes,No,No,No
1,No,No,No,No,Yes,No,No,No,No,No,...,Yes,Not applicable,No,No,Yes,Yes,Yes,No,No,No
2,No,No,Yes,No,Yes,Not applicable,No,No,No,Not applicable,...,Yes,Not applicable,No,Not applicable,No,Not applicable,Yes,No,No,No
3,No,No,No,No,Yes,No,No,No,No,Not applicable,...,Yes,Not applicable,No,Not applicable,No,Not applicable,No,No,No,No
4,No,No,No,No,Yes,No,Yes,Yes,No,Not applicable,...,Yes,Not applicable,Yes,Not applicable,No,Not applicable,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30075,No,No,No,No,Yes,Yes,No,No,No,No,...,Yes,Not applicable,No,Not applicable,No,Not applicable,Yes,No,NaN,No
30076,No,No,No,No,Yes,No,No,No,No,Not applicable,...,No,No,No,Not applicable,No,Not applicable,No,No,NaN,No
30077,No,No,No,No,Yes,Not applicable,No,No,No,Not applicable,...,No,No,No,No,Yes,No,Yes,No,NaN,No
30078,No,No,No,No,Yes,No,No,No,No,No,...,No,No,No,Not applicable,No,Not applicable,Not eligible to vote,Not applicable,NaN,No


In [52]:
k = df_raw.apply(lambda col : pd.Series([".a"]).isin(col)).T[0]

In [55]:
df[k.index]

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v263,v264,v265,v266,v267,v268,v269,v270,cntry,satisfied
0,9948,2,2,74,11010,.a,2,2,2,2,...,.a,.a,.a,.a,.a,.a,.a,NaN,AT,0
1,25601,4,2,58,11010,.a,2,2,2,2,...,.a,.a,.a,.a,.a,.a,.a,NaN,AT,0
2,8592,6,2,47,11010,11010,2,2,1,2,...,1963,1993,1995,.a,.a,.a,.a,NaN,AT,1
3,29593,10,2,22,11010,.a,2,2,2,2,...,.a,.a,.a,.a,.a,.a,.a,NaN,AT,0
4,4252,0,1,24,11010,.a,2,2,2,2,...,.a,.a,.a,.a,.a,.a,.a,NaN,AT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30075,34440,0,1,72,14120,.a,2,2,2,2,...,1945,.a,.a,.a,.a,.a,.a,.a,SI,1
30076,13566,0,1,38,14120,.a,2,2,2,2,...,.a,.a,.a,.a,.a,.a,.a,.a,SI,1
30077,29824,5,2,49,14120,.a,2,2,2,2,...,1967,1990,1993,.a,.a,.a,.a,.a,SI,0
30078,9573,0,1,16,14120,.a,2,2,2,2,...,1972,1971,.a,.a,.a,.a,.a,.a,SI,1


Index(['id', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9',
       ...
       'v263', 'v264', 'v265', 'v266', 'v267', 'v268', 'v269', 'v270', 'cntry',
       'satisfied'],
      dtype='object', length=273)

In [78]:
k = df.apply(lambda col : pd.Series([".a"]).isin(col) | pd.Series([".b"]).isin(col)).T[0]

In [79]:
numeric_multinomial = list(k[k].index)

In [80]:
df.apply(pd.to_numeric, args=('coerce',)).dropna(axis=1).columns

Index(['id', 'v15', 'v16', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32',
       'v33', 'v34', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v44',
       'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v54', 'v55',
       'v106', 'v107', 'v129', 'v134', 'v175', 'v176', 'v185', 'v217', 'v218',
       'v244', 'v245', 'v246', 'v247', 'satisfied'],
      dtype='object')

In [81]:
numeric_continuous = ['v15', 'v16', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32',
       'v33', 'v34', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v44',
       'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v54', 'v55',
       'v106', 'v107', 'v129', 'v134', 'v175', 'v176', 'v185', 'v217', 'v218',
       'v244', 'v245', 'v246', 'v247']

In [82]:
categorical = list(lookup_df[lookup_df["type"] != "numeric (double)"]["name"])

In [83]:
len(numeric_continuous)

42

In [84]:
len(numeric_multinomial)

219

In [85]:
len(categorical)

7

In [88]:
numeric_multinomial

['v1',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9',
 'v10',
 'v11',
 'v12',
 'v13',
 'v14',
 'v18',
 'v19',
 'v21',
 'v22',
 'v23',
 'v24',
 'v35',
 'v43',
 'v53',
 'v56',
 'v57',
 'v58',
 'v59',
 'v60',
 'v61',
 'v62',
 'v63',
 'v64',
 'v65',
 'v66',
 'v67',
 'v68',
 'v69',
 'v70',
 'v71',
 'v72',
 'v73',
 'v74',
 'v75',
 'v76',
 'v77',
 'v79',
 'v80',
 'v81',
 'v82',
 'v83',
 'v84',
 'v85',
 'v86',
 'v87',
 'v88',
 'v89',
 'v90',
 'v91',
 'v92',
 'v93',
 'v94',
 'v95',
 'v96',
 'v97',
 'v98',
 'v99',
 'v100',
 'v101',
 'v102',
 'v103',
 'v104',
 'v105',
 'v108',
 'v109',
 'v110',
 'v111',
 'v112',
 'v113',
 'v114',
 'v115',
 'v116',
 'v117',
 'v118',
 'v119',
 'v120',
 'v121',
 'v122',
 'v124',
 'v125',
 'v126',
 'v127',
 'v128',
 'v130',
 'v131',
 'v133',
 'v135',
 'v136',
 'v137',
 'v138',
 'v139',
 'v140',
 'v141',
 'v142',
 'v143',
 'v144',
 'v145',
 'v146',
 'v147',
 'v148',
 'v149',
 'v150',
 'v151',
 'v152',
 'v153',
 'v156',
 'v157',
 'v158',
 'v159',
 'v160',
 'v1

In [86]:
set(df.columns).difference(numeric_continuous).difference(numeric_multinomial).difference(categorical)

{'cntry', 'id', 'satisfied', 'v123', 'v132'}

In [89]:
categorical

['v17', 'v20', 'v25', 'v78', 'v154', 'v155', 'v161']

In [90]:
len(categorical) + len(numeric_continuous) + len(numeric_multinomial)

268

In [98]:
df[numeric_continuous].nunique()

v15      2
v16      2
v26      2
v27      2
v28      2
v29      2
v30      2
v31      2
v32      2
v33      2
v34      2
v36      2
v37      2
v38      2
v39      2
v40      2
v41      2
v42      2
v44      2
v45      2
v46      2
v47      2
v48      2
v49      2
v50      2
v51      2
v52      2
v54      2
v55      2
v106     2
v107     2
v129    12
v134     2
v175     2
v176     2
v185     4
v217     2
v218     2
v244     2
v245     2
v246     2
v247     2
dtype: int64

In [93]:
numeric_continous

NameError: name 'numeric_continous' is not defined

In [100]:
df["v64"].value_counts()

12    4746
11    3145
13    2941
14    2283
15    2227
16    2150
10    1971
9     1791
17    1689
8     1519
18    1329
19     695
20     640
7      575
6      435
4      395
.b     238
21     216
22     209
5      174
3      124
0      114
23     114
24      72
25      70
2       58
.a      30
1       27
30      22
26      20
27      16
.c      11
32       8
28       6
29       6
36       2
38       2
40       2
35       2
50       2
34       1
47       1
45       1
37       1
Name: v64, dtype: int64

In [11]:
codebook_df[codebook_df["type"].str.startswith("string")]

,name,desc,type,label,range,units,unique values,missing .,unique mv codes,missing .*,"missing """"",mean,std. dev
16,v17,Country of birth,string (str2),NaN,NaN,NaN,172,NaN,NaN,NaN,"0/39,325",NaN,NaN
19,v20,Region,string (str5),NaN,NaN,NaN,251,NaN,NaN,NaN,"0/39,325",NaN,NaN
24,v25,Citizenship,string (str2),NaN,NaN,NaN,130,NaN,NaN,NaN,"0/39,325",NaN,NaN
77,v78,"Country of birth, father",string (str2),NaN,NaN,NaN,171,NaN,NaN,NaN,"0/39,325",NaN,NaN
153,v154,Language most often spoken at home: first ment...,string (str3),NaN,NaN,NaN,123,NaN,NaN,NaN,"0/39,325",NaN,NaN
154,v155,Language most often spoken at home: second men...,string (str3),NaN,NaN,NaN,129,NaN,NaN,NaN,"0/39,325",NaN,NaN
160,v161,"Country of birth, mother",string (str2),NaN,NaN,NaN,173,NaN,NaN,NaN,"0/39,325",NaN,NaN


In [6]:
categorical = list(codebook_df[codebook_df["type"].str.startswith("string")]["name"])

In [32]:
categorical

['v17', 'v20', 'v25', 'v78', 'v154', 'v155', 'v161']

In [28]:
print(df[categorical[4]].value_counts().to_string())

GER    3619
ENG    3491
FRE    1699
CZE    1658
LIT    1550
FIN    1536
POL    1431
DUT    1421
SWE    1404
HEB    1380
SPA    1307
HUN    1284
DAN    1151
POR    1018
NOR    1017
EST    1008
SLV     926
RUS     907
GSW     687
ARA     454
999     177
CAT     142
TUR      71
ITA      67
GLG      57
888      55
SRP      46
BOS      35
ALB      32
FRS      32
RUM      28
HRV      23
KUR      23
BAQ      23
URD      20
PER      18
AMH      16
APA      16
ROA      15
CHI      12
777      11
HIN      11
GLE      10
SOM      10
ARM      10
IRA       9
SLO       9
TAM       8
GUJ       8
GEM       8
ROM       7
FRM       6
GRE       6
CPP       6
WEL       5
VIE       5
BUL       5
MIS       4
YID       4
BER       4
UKR       4
PAP       4
THA       3
SLA       3
BEN       3
LAV       3
MAL       3
TGL       3
PAN       2
MAY       2
NEP       2
ROH       2
TIR       2
MLG       2
AZE       2
UZB       2
TGK       2
MAN       2
SYR       2
IND       1
AKA       1
ABK       1
LIN       1
TAI 

In [31]:
print(df[['v17', 'v78', 'v161']].to_string())

      v17 v78 v161
0      66  66   66
1      66  66   66
2      66  66   66
3      66  66   66
4      66  66   66
5      66  66   66
6      66  66   66
7      HU  HU   HU
8      GB  GB   GB
9      66  66   66
10     66  66   66
11     66  66   66
12     66  HR   66
13     66  HR   BA
14     66  66   66
15     66  66   DE
16     66  66   66
17     66  66   66
18     66  66   66
19     66  66   66
20     66  66   66
21     66  66   66
22     66  66   66
23     66  66   66
24     66  66   66
25     66  66   66
26     66  66   66
27     66  66   66
28     66  66   66
29     66  66   66
30     66  66   66
31     66  66   66
32     66  66   66
33     66  66   66
34     RS  RS   RS
35     66  66   66
36     66  66   66
37     IR  IR   IR
38     HR  HR   HR
39     66  66   66
40     66  66   66
41     66  66   66
42     66  66   66
43     66  66   66
44     PL  PL   PL
45     66  66   66
46     66  66   66
47     66  66   66
48     66  66   66
49     66  66   66
50     66  66   66
51     66  6